In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()
API_TOKEN = str(os.getenv('API_TOKEN'))

In [2]:
def read_json_file(file_path: str=None, url: str=None) -> dict:

    if url is not None:
        response = requests.get(url)
        data = response.json()
        return data

    elif file_path is not None:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            return data

In [6]:
# path = 'https://raw.githubusercontent.com/Ciencia-de-Dados-IMT-2023/Datasets/main/spotify_million_playlist_dataset/mpd.slice.0-999.json'
path = "../data/00 - Raw Data/mpd.slice.3000-3999.json"
data = read_json_file(file_path=path)

In [8]:
df_musics = pd.DataFrame()

# Iterating over playlists
for playlist in data['playlists']:
    name_playlist = playlist['name']
    id_playlist = playlist['pid']

    print(f'{id_playlist} - Playlist: {name_playlist}')

    # Iterating over the songs in the playlist
    musics_playlist = playlist['tracks']

    for music in musics_playlist:
        name_musica = music['track_name']
        id_musica = music['track_uri']

        # print(f'\tSong: {name_song} - ID: {song_id}')
        df_musica_aux = pd.DataFrame.from_dict(music, orient='index')
        df_musica_aux = df_musica_aux.T

        # Add the playlist name and playlist ID
        df_musica_aux['playlist_name'] = name_playlist
        df_musica_aux['playlist_id'] = id_playlist

        df_musica_aux = df_musica_aux.T
        df_musics = pd.concat([df_musics, df_musica_aux], axis=1)
        

df_musics = df_musics.T
df_musics.head(5)
df_musics.to_csv(f'../data/01 - Extracted Data/df_musics_3000_3999.csv', index=False)

3000 - Playlist: Birthday Party
3001 - Playlist: Chill
3002 - Playlist: Run For Your Life
3003 - Playlist: chill
3004 - Playlist: riding
3005 - Playlist: Dad
3006 - Playlist: chillin'
3007 - Playlist: Hard Rock
3008 - Playlist: Blessed 
3009 - Playlist: CB
3010 - Playlist: 90'
3011 - Playlist: summer vibes
3012 - Playlist: 2013
3013 - Playlist: rewind
3014 - Playlist: November 2016
3015 - Playlist: Upbeat
3016 - Playlist: Workout
3017 - Playlist: Training.
3018 - Playlist: Romeo
3019 - Playlist: Hindi songs
3020 - Playlist: 2017
3021 - Playlist: Get down
3022 - Playlist: Josh Garrels
3023 - Playlist: The Playlist
3024 - Playlist: Reggeaton
3025 - Playlist: country
3026 - Playlist: skiing
3027 - Playlist: HYPE
3028 - Playlist: rap
3029 - Playlist: Country
3030 - Playlist: THROW IT BACK 
3031 - Playlist: Somo
3032 - Playlist: SAD
3033 - Playlist: Have Yourself a Merry Little Christmas
3034 - Playlist: Shelby
3035 - Playlist: party
3036 - Playlist: My Music
3037 - Playlist: The Day
3038 -

In [9]:
df_musics = df_musics[['track_uri', 'artist_uri', 'album_uri']]

df_musics['track_uri'] = df_musics['track_uri'].str.replace('spotify:track:', '')
df_musics['artist_uri'] = df_musics['artist_uri'].str.replace('spotify:artist:', '')
df_musics['album_uri'] = df_musics['album_uri'].str.replace('spotify:album:', '')

df_musics.head(5)

C:\Users\guisa\AppData\Local\Temp\ipykernel_21556\3640793233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_musics['track_uri'] = df_musics['track_uri'].str.replace('spotify:track:', '')
C:\Users\guisa\AppData\Local\Temp\ipykernel_21556\3640793233.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_musics['artist_uri'] = df_musics['artist_uri'].str.replace('spotify:artist:', '')
C:\Users\guisa\AppData\Local\Temp\ipykernel_21556\3640793233.py:5: SettingWithCopyWarning: 
A value is trying to be set

,track_uri,artist_uri,album_uri
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K
1,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,0z7pVBGOD7HCIB7S8eLkLI
2,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,25hVFAxTlDvXbx2X2QkUkE
3,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,6QPkyl04rXwTGlGlcYaRoW
4,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,6NmFmPX56pcLBOFMhIiKvF


In [10]:
# scripts/api_spotify.py

def make_req(route: str, id: str) -> requests.models.Response:
    url = f'https://api.spotify.com/v1/{route}/{id}'

    headers = {
        'Authorization': f'Bearer {API_TOKEN}'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f'Erro: {response.status_code}')
        print(f'Erro: {response.json()}')
        import sys
        sys.exit(1)

    else:
        return response


def get_track(id_track: str) -> pd.DataFrame:
    data = make_req('tracks', id_track).json()
    df = pd.DataFrame(data['tracks'])
    return df


def get_audio_features(id_track: str) -> dict:
    data = make_req('audio-features', id_track).json()
    df = pd.DataFrame(data['audio_features'])
    return df


def get_album(id_album: str) -> dict:
    data = make_req('albums', id_album).json()
    df = pd.DataFrame(data['albums'])
    return df


def get_artist_data(id_artist: str) -> dict:
    data = make_req('artists', id_artist).json()
    df = pd.DataFrame(data['artists'])
    return df



In [16]:
def get_music_features(tracks_ids: list) -> dict:

    tracks_ids_str = ','.join(tracks_ids)
    tracks_ids_str = '?ids=' + tracks_ids_str

    # 1. Get track data
    tracks_list_df = get_track(tracks_ids_str)

    # Collect album ids
    tracks_list_df['id_album'] = tracks_list_df['album'].apply(lambda x: x['id'])

    # Collect artist ids
    tracks_list_df['id_artist'] = tracks_list_df['artists'].apply(lambda x: [d['id'] for d in x])

    # Expand artist ids to separate rows
    tracks_list_df = tracks_list_df.explode('id_artist')

    
    # 2. Get audio features
    df_audio_features = pd.DataFrame()
    for i in range(0, len(tracks_ids), 50):
        tracks_ids_str = ','.join(tracks_ids[i:i+50] )
        tracks_ids_str = '?ids=' + tracks_ids_str
        df_audio_features_aux = get_audio_features(tracks_ids_str)
        df_audio_features = pd.concat([df_audio_features, df_audio_features_aux], axis=0)

    # Filtering columns
    df_audio_features = df_audio_features[['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    
    # Merge audio features with track data
    tracks_list_df = tracks_list_df.merge(df_audio_features, on='id', how='left')


    # 3. Get album data
    df_albums = pd.DataFrame()
    albums_ids = tracks_list_df['id_album'].unique().tolist()
    for i in range(0, len(albums_ids), 20):
        albums_ids_str = ','.join(albums_ids[i:i+20] )
        albums_ids_str = '?ids=' + albums_ids_str
        df_albums_aux = get_album(albums_ids_str)
        df_albums = pd.concat([df_albums, df_albums_aux], axis=0)

    # Filtering columns
    df_albums = df_albums[['id', 'name', 'popularity', 'release_date', 'total_tracks', 'type']]

    # Rename columns
    df_albums = df_albums.rename(columns={'id': 'id_album', 'name': 'album_name', 'popularity': 'album_popularity', 'release_date': 'album_release_date', 'total_tracks': 'album_total_tracks', 'type': 'album_type'})

    # Merge album data with track data
    tracks_list_df = tracks_list_df.merge(df_albums, on='id_album', how='left')
    

    # 4. Get artist data
    df_artists = pd.DataFrame()
    artists_ids = tracks_list_df['id_artist'].unique().tolist()
    for i in range(0, len(artists_ids), 50):
        artists_ids_str = ','.join(artists_ids[i:i+50] )
        artists_ids_str = '?ids=' + artists_ids_str
        df_artists_aux = get_artist_data(artists_ids_str)
        df_artists = pd.concat([df_artists, df_artists_aux], axis=0)

    # Filtering columns
    df_artists = df_artists[['id', 'name', 'genres', 'popularity', 'type', 'followers']]

    # Rename columns
    df_artists = df_artists.rename(columns={'id': 'id_artist', 'name': 'artist_name', 'genres': 'artist_genres', 'popularity': 'artist_popularity', 'type': 'artist_type', 'followers': 'artist_followers'})

    # Merge artist data with track data
    tracks_list_df = tracks_list_df.merge(df_artists, on='id_artist', how='left')

    # Collect number os followers
    tracks_list_df['artist_followers'] = tracks_list_df['artist_followers'].apply(lambda x: x['total'])

    # Expand artist ids to separate rows
    tracks_list_df = tracks_list_df.explode('artist_genres')

    # Keep only relevant artist_genres
    relevant_artist_genres = ['pop', 'hip hop', 'r&b', 'rap', 'reggae', 'rock', 'punk', 'alternative']
    tracks_list_df = tracks_list_df[tracks_list_df['artist_genres'].isin(relevant_artist_genres)]

    tracks_list_df = tracks_list_df[[
        'id', 'id_album', 'id_artist', 'name', 'explicit', 'duration_ms', 'popularity', # Track data
        'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', # Audio features
        'album_name', 'album_popularity', 'album_release_date', 'album_total_tracks', 'album_type', # Album data
        'artist_name', 'artist_genres', 'artist_popularity', 'artist_type', 'artist_followers' # Artist data
        ]]

    return tracks_list_df


In [13]:
tracks_ids = df_musics['track_uri'].unique().tolist()

In [19]:
df_musics = pd.DataFrame()
print(f'There are {len(tracks_ids)} tracks to be processed')

for i in range(0, len(tracks_ids), 50):
    print(f'Processing tracks {i} to {i+50}')
    df_aux = get_music_features(tracks_ids[i:i+50])
    df_musics = pd.concat([df_musics, df_aux], axis=0)

There are 34443 tracks to be processed
Processing tracks 0 to 50
Processing tracks 50 to 100
Processing tracks 100 to 150
Processing tracks 150 to 200
Processing tracks 200 to 250
Processing tracks 250 to 300
Processing tracks 300 to 350
Processing tracks 350 to 400
Processing tracks 400 to 450
Processing tracks 450 to 500
Processing tracks 500 to 550
Processing tracks 550 to 600
Processing tracks 600 to 650
Processing tracks 650 to 700
Processing tracks 700 to 750
Processing tracks 750 to 800
Processing tracks 800 to 850
Processing tracks 850 to 900
Processing tracks 900 to 950
Processing tracks 950 to 1000
Processing tracks 1000 to 1050
Processing tracks 1050 to 1100
Processing tracks 1100 to 1150
Processing tracks 1150 to 1200
Processing tracks 1200 to 1250
Processing tracks 1250 to 1300
Processing tracks 1300 to 1350
Processing tracks 1350 to 1400
Processing tracks 1400 to 1450
Processing tracks 1450 to 1500
Processing tracks 1500 to 1550
Processing tracks 1550 to 1600
Processing t

SystemExit: 1

c:\Users\guisa\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [132]:
%timeit
df_musics.iloc[0]

id                                        0UaMYEvWZi0ZqiDOoHU3YI
id_album                                  6vV5UrXcfyQD1wu4Qo2I9K
id_artist                                 2wIVse2owClT7go1WT98tk
name                  Lose Control (feat. Ciara & Fat Man Scoop)
explicit                                                    True
duration_ms                                               226863
popularity                                                    69
acousticness                                              0.0311
danceability                                               0.904
energy                                                     0.813
instrumentalness                                         0.00697
key                                                            4
liveness                                                  0.0471
loudness                                                  -7.105
mode                                                           0
speechiness              

In [133]:
df_musics['id'].nunique()

484